### Passo 1 - Importar Arquivos e Bibliotecas

In [2]:
# Bibliotecas
import pandas as pd
import pathlib
import win32com.client as win32
outlook = win32.Dispatch('outlook.application')

In [3]:
# Arquivos
df_email = pd.read_excel(r'Bases de Dados/Emails.xlsx')
df_lojas = pd.read_csv(r'Bases de Dados/Lojas.csv', encoding='latin1', sep=';')
df_vendas = pd.read_excel(r'Bases de Dados/Vendas.xlsx')

display(df_vendas.head())


,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,1,Sapato Estampa,1,358,358
1,1,2019-01-01,1,Camiseta,2,180,360
2,1,2019-01-01,1,Sapato Xadrez,1,368,368
3,2,2019-01-02,3,Relógio,3,200,600
4,2,2019-01-02,3,Chinelo Liso,1,71,71


### Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [4]:
vendas = df_vendas.merge(df_lojas, on='ID Loja')
display(vendas)

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja
0,1,2019-01-01,1,Sapato Estampa,1,358,358,Iguatemi Esplanada
1,1,2019-01-01,1,Camiseta,2,180,360,Iguatemi Esplanada
2,1,2019-01-01,1,Sapato Xadrez,1,368,368,Iguatemi Esplanada
3,21,2019-01-02,1,Camisa Gola V Listrado,2,116,232,Iguatemi Esplanada
4,34,2019-01-02,1,Sapato Listrado,1,363,363,Iguatemi Esplanada
...,...,...,...,...,...,...,...,...
100994,69846,2019-12-25,8,Short Estampa,2,96,192,Salvador Shopping
100995,69846,2019-12-25,8,Tênis Estampa,5,256,1280,Salvador Shopping
100996,69850,2019-12-25,8,Calça Estampa,4,177,708,Salvador Shopping
100997,69972,2019-12-26,8,Terno Liso,3,720,2160,Salvador Shopping


In [5]:
dic_lojas = {}
for loja in df_lojas['Loja']:
    dic_lojas[loja] = vendas.loc[vendas['Loja'] == loja, :]


In [6]:
dia_indicador = vendas['Data'].max()

print(dia_indicador)

2019-12-26 00:00:00


### Passo 3 - Salvar a planilha na pasta de backup

In [7]:
caminho_backup = pathlib.Path(r'Backup Arquivos Lojas')

arquivos_backup = caminho_backup.iterdir()

#Usamos a lista para verificar se o arquivo da loja existe dentro da lista
lista_backup = [arquivo.name for arquivo in arquivos_backup]

for loja in dic_lojas:
    if loja not in lista_backup:
        nova_pasta = caminho_backup / loja
        nova_pasta.mkdir()
    nome_arquivo = '{}-{}_{}.xlsx'.format(dia_indicador.month, dia_indicador.year, loja)
    local_arquivo = caminho_backup / loja / nome_arquivo
    
    dic_lojas[loja].to_excel(local_arquivo)

### Passo 4 - Calcular o indicador para 1 loja

In [21]:
loja = 'Norte Shopping'
loja_vendas = dic_lojas[loja]
vendas_loja_dia = loja_vendas.loc[loja_vendas['Data'] == dia_indicador, :]

#Faturamento
faturamento_ano = loja_vendas['Valor Final'].sum()
faturamento_dia = vendas_loja_dia['Valor Final'].sum()
print(f'O faturamento do ano foi de R$ {faturamento_ano}')
print(f'O faturamento do dia foi de R$ {faturamento_dia}')

#Diversidade de produtos
qtd_produtos_ano = len(loja_vendas['Produto'].unique())
print(f'{qtd_produtos_ano} produtos direntes vendidos no ano')

qtd_produtos_dia = len(vendas_loja_dia['Produto'].unique())
print(f'{qtd_produtos_dia} produtos diferentes vendidos no dia')

#ticket medio 

valor_venda = loja_vendas.groupby('Código Venda').sum()

display(valor_venda)

O faturamento do ano foi de R$ 1711968
O faturamento do dia foi de R$ 1259
120 produtos direntes vendidos no ano
2 produtos diferentes vendidos no dia


,ID Loja,Quantidade,Valor Unitário,Valor Final
Código Venda,,,,
2,6,4,271,671
7,6,2,221,221
73,6,6,127,367
137,3,2,50,100
147,9,7,247,544
...,...,...,...,...
69845,9,3,623,623
69888,3,1,294,294
69916,3,2,700,1400


### Passo 5 - Enviar por e-mail para o gerente

### Passo 6 - Automatizar todas as lojas

### Passo 7 - Criar ranking para diretoria

### Passo 8 - Enviar e-mail para diretoria